# Retrieving Information from the PDB using the Web API

``````{admonition} Overview
:class: overview

Questions: 

* What is a REST API?

* How can I use Python to retrieve data from a REST API?

Objectives:

* Manipulate URLs to use the PDB REST API.

* Use Python requests to retrieve information from the PDB.

``````

In this lesson, we will explore how to retrieve information from the PDB Web API.

Many databases can be accessed programmatically through something called a REST API. REST stands for Representational State Transfer. API stands for Application Programming Interface. A REST API is a type of web API that is used to allow different software systems to communicate with each other over the internet.

Usually a REST API is accessed by varying parameters in a web address.

We will work with two types of API from the PDB in this lesson. 

## File Download using Biopython

Biopython has functions for retrieving structure files.
To download a structure file from the PDB, you make a `PDBList`.
Then, you use the function `pdb_list.retrieve_pdb_file.
This function takes the PDB ID as a parameter. 
You specify the folder where you want the file saved using `pdir=FOLDER`. 
If we want to download the structure as an MMCIF, we have to add `file_format="mmCif"`.

For example, the following cell downloads the PDB `4HHB`, hemoglobin.

In [ ]:
from Bio.PDB import PDBList

# Create an instance of the PDBList class
pdb_list = PDBList()

# Specify the PDB ID of the structure you want to download
pdb_id = "4hhb" #zinc finger

# Download the MMCIF file using the retrieve_pdb_file method
pdb_filename = pdb_list.retrieve_pdb_file(pdb_id, pdir="data/PDB_files", file_format="mmCif")

# Print the name of the downloaded file
print(pdb_filename)

If you check your data folder, you will see that you now have a file called `4hhb.cif`.

## PDB Data API

You can use the web to retrieve information about a molecule using its PDB ID using the PDB Data API. 
This will give you access to information about the PDB entry, rather than the structure file like with the Biopython file download.

As mentioned above, web APIs often work by varying text in a web address (also called URL).

The PDB Data API has the following format:

```
https://data.rcsb.org/rest/v1/core/<TYPE_OF_RESOURCE>/<IDENTIFIER>
```

For example, to get the full entry for `4hhb`, you would do:

https://data.rcsb.org/rest/v1/core/entry/4hhb

Try clicking this link! It will display text in your browser with info about `4HHB`.

There are many things you can do with the REST API that are beyond this workshop. 
However, one interesting thing you can do is to retrieve PubMed annotations for an entry's primary citation.

https://data.rcsb.org/rest/v1/core/pubmed/4hhb

The data in these results is in a data format called JSON.  This is a commonly used type of return format for web APIs because it can be processed using programming. 
Its format is similar to a Python dictionary, having keys and values.

## Programmatic Access of APIs
REST APIs start being more useful when you access them programmatically. We are going to use Python to retrieve the data at the URL and convert it to a format we can work with.

We will use a Python library called requests. Requests is used to retrieve information from websites and URLs.

In [ ]:
import requests

To get information from a URL, we use the requests.get method. The argument to this function is the URL we'd like to retrieve information from.

In [ ]:
data = requests.get("https://data.rcsb.org/rest/v1/core/entry/4hhb")

Our data variable now contains the results and other information about the request we made.

If our request was successful. It will have a status code of `200`. 
This is a feature of web URLs. 
If they are successfully retrieved, they send a status code of 200. 
This is true for any website you receive.
One status code you may be familiar with is `404`, which occurs when a resources is not found.

Note that `status_code` is not a function associated with data, so we do not need to use `()`. 

In [ ]:
data.status_code

We can see the JSON associated with our request by calling the .json() method, which we will save in a variable called `info_4hhb`. Our variable is now similar to a Python dictionary, which is a data type that has key, value pairs.

In [ ]:
info_4hhb = data.json()

Once we have loaded our returned values as json, we can see all of the keywords associated with an entry using variable.keys()



In [ ]:
info_4hhb.keys()

The variable `info_4hhb` is a Python dictionary, meaning that
we can access information in the dictionary using the syntax:

```python
dictionary_name["key_name"]
```

In [ ]:
info_4hhb["exptl"]

In [ ]:
info_4hhb["struct_keywords"]

In [ ]:
info_4hhb["struct"]

In [ ]:
info_4hhb["rcsb_entry_info"]

You can also use the PDB Data API to retrieve information about interfaces between polymeric entities (protein, or nucleic acids) using
one of the API endpoints by changing the URL.

The format for querying about interfaces is

```
https://data.rcsb.org/rest/v1/core/interface/<pdb_id>/<assembly_id>/<interface_id>
```

In the cell below, we get the first interface for assembly 1 (there is only one assembly in this PDB entry).

In [ ]:
interface = requests.get("https://data.rcsb.org/rest/v1/core/interface/4hhb/1/1")

In [ ]:
interface.status_code

In [ ]:
interface_info = interface.json()
interface_info["rcsb_interface_info"]

Try changing the numbers in the URL. How does it change your results?

## PDB Search API

The API we were just working with is called the PDB "Data" API. 

However, the PDB has another API called the "search" API that let's you search based on keywords, host species, sequence similarity, and many other things.
The search API can be complicated, but is [well documented](https://search.rcsb.org/#search-api).

The format for this URL is:

`https://search.rcsb.org/rcsbsearch/v2/query?json={search-request}`

Where `search-request` is a JSON (similar to a Python dictionary) containing your search parameters.
We will create a Python dictionary, then use the `json` library to convert it to a json (the data type required by the search API).


In [ ]:
import json

my_query = {
  "query": {
    "type": "terminal",
    "service": "full_text",
    "parameters": {
        "value": '"oxygen storage"'
    }
  },
  
  "return_type": "entry"
}

my_query = json.dumps(my_query)


Now, we use `requests.get` with the our search query and the search API URL format.

In [ ]:
data = requests.get(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_query}")
results = data.json()
results

Our results tell us that our query returned `619` results (`total_count`).
The `results_set` keyword has a list of results. 
The `identifier` key in each value in `results_set` tells
us the PDB ID of a search result.

You will notice that even though the search has told us there are `619` results, we only have 10 values in our results variable.
This is because the API always counts the number of results, but will only return 10 unless we ask for more.

We could now combine this with Biopython or the data API to get information about the structures.
For example, we will retrieve the title of the paper where this structure was published we could do so:

In [ ]:
first_result = results["result_set"][0]["identifier"]
print(first_result)

In [ ]:
data = requests.get(f"https://data.rcsb.org/rest/v1/core/entry/{first_result}")
result = data.json()
result["struct"]

You can also control the format of the returned results including the
number of results and how the results are sorted.
The query below searchs for the phrase "oxygen storage" in the keywords of the structure. 
We've specified that we want 50 values returned (instead of the usual 10), and for the results to be sorted
by the initial release date in ascending order (oldest result first).
If you wanted to see the 10 most recently released results, you would change 
"asc" to "desc" in the query below.

In [ ]:
my_query = {
  "query": {
    "type": "terminal",
    "service": "text",
    "parameters": {
        "attribute": "struct_keywords.pdbx_keywords",
        "operator": "contains_phrase",
        "value": '"oxygen storage"'
    }
  },

  "request_options": {
    "paginate": {
      "start": 0,
      "rows": 50,
    },
    "sort": [
      {
      "sort_by": "rcsb_accession_info.initial_release_date",
      "direction": "asc" 
    }
    ]
  },
  
  "return_type": "entry"
}

my_query = json.dumps(my_query)

Now, we use `requests.get` with the our search query and the search API URL format.


In [ ]:
data = requests.get(f"https://search.rcsb.org/rcsbsearch/v2/query?json={my_query}")

In [ ]:
data.status_code

In [ ]:
results = data.json()
results

## Using Biopython to Analyze Search Results

The API becomes really interesting when we use the results for analysis.
Keeping our example of oxygen storage, we could use Biopython to analyze the structures we have retrieved to see
if there are heme groups in the structures and if there are common motifs in the binding of the heme groups to proteins.

The following cells show a farily complicated analysis of the structures we have retrieved using Biopython in the cells just above this section.
If you are new to programming, the next few cells will be hard to understand.
However, they are a demonstration of the type of analysis that can be done by retrieving search results programmatically
and analyzing data using Python.
The end of the analysis prints the most commonly found neighboring residues to iron in the structures retrieved by the search.

The program has multiple steps:

1. Biopython is used to download the mmcif files for all of the structures.
2. Biopython is used to create structure objects, sort through atoms, and find residues neighboring iron atoms.
3. Python functions (particular [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) ) are used to count residue combinations.

```python
## Step 1
from Bio.PDB import PDBList

# Create an instance of the PDBList class
pdb_list = PDBList()

# Download all of the structure files
for result in results["result_set"]:
    pdb_id = result["identifier"].lower()

    # Download the MMCIF file using the retrieve_pdb_file method
    pdb_filename = pdb_list.retrieve_pdb_file(pdb_id, pdir="pdb_files", file_format="mmCif")


## Step 2

from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB import NeighborSearch
from collections import Counter

# Create an MMCIFParser object to parse mmCIF files.
parser = MMCIFParser(QUIET=True)

# Define the maximum distance (in Ångströms) for identifying neighboring residues.
cutoff_distance = 5

# Initialize a dictionary to store the neighboring residues for each protein structure.
residue_neighbors = {}

# The 'results' variable is a dictionary containing search results.
# Each result in 'results["result_set"]' represents a protein structure with a PDB ID.
for result in results["result_set"]:
    # Extract the PDB ID and convert it to lowercase.
    pdb_id = result["identifier"].lower()

    # Parse the mmCIF file for the protein structure using the PDB ID.
    # The file is expected to be located in the 'pdb_files' directory.
    structure = parser.get_structure(pdb_id, f"pdb_files/{pdb_id}.cif")
    
    # Extract all atoms from the protein structure.
    atoms = list(structure.get_atoms())
    
    # Create a NeighborSearch object to perform neighbor searches.
    neighbor_search = NeighborSearch(atoms)
    
    # Initialize a list to store the neighboring residues for this protein structure.
    neighbor_list = []

    # Loop through the atoms in the protein structure.
    for atom in atoms:
        # Check if the atom is an iron (Fe) atom.
        if atom.element == "FE":
            # Get the parent residue of the iron atom.
            iron_residue = atom.get_parent()

            # Find atoms within the cutoff distance from the iron atom.
            neighbors = neighbor_search.search(atom.get_coord(), cutoff_distance)
            
            # Loop through the neighboring atoms.
            for neighbor in neighbors:
                # Get the parent residue of the neighboring atom.
                residue = neighbor.get_parent()
    
                # Check if the neighboring residue is different from the iron-containing residue.
                if residue != iron_residue:
                    # Add the neighboring residue to the list.
                    neighbor_list.append(residue)
                    
    # Store the unique neighboring residues in the dictionary using the PDB ID as the key.
    residue_neighbors[pdb_id] = set(neighbor_list)

# The 'residue_neighbors' dictionary contains the neighboring residues for each protein structure.


## Step 3
## Now we will want to count the residue neighbor types.
# Initialize an empty Counter object to store the counts of residue combinations.
combination_counts = Counter()

# Iterate over the items in the 'residue_neighbors' dictionary.
# Each item consists of a PDB ID ('pdb_id') and a set of neighboring residues ('neighbors') to iron atoms.
for pdb_id, neighbors in residue_neighbors.items():
    # Extract the residue names ('resname') for each neighboring residue using the 'get_resname' method.
    resname = [x.get_resname() for x in neighbors if x.get_resname()]
    
    # Count the occurrences of each residue name in the current combination.
    res_counts = Counter(resname)
    
    # Convert the residue counts to a tuple of (residue, count) pairs, sorted by residue name.
    # This standardizes the representation of each combination.
    combination = tuple(sorted(res_counts.items()))
    
    # Update the combination_counts with the current combination.
    combination_counts.update([combination])

# Use the 'most_common' method to get the most common residue combinations.
# The result is a list of tuples, where each tuple contains a combination and its count.
most_common_combinations = combination_counts.most_common()

# For example, to get the top 5 most common combinations, use 'most_common(5)'.
top_5_combinations = combination_counts.most_common(5)
print("\nTop 5 most common residue combinations for iron neighbors:")
for combination, count in top_5_combinations:
    combination_str = ', '.join([f"{count} {residue}" for residue, count in combination])
    print(f"Combination: {combination_str}, Count: {count}")
```

```output

Top 5 most common residue combinations for iron neighbors:
Combination: 2 HIS, 1 HOH, 1 VAL, Count: 4
Combination: 1 CMO, 2 HIS, 1 VAL, Count: 3
Combination: 2 HIS, 1 HOH, 1 OXY, 1 VAL, Count: 2
Combination: 5 HIS, 2 HOH, 2 VAL, Count: 2
Combination: 4 HIS, 2 HOH, 2 VAL, Count: 2
```